In [ ]:
pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import numpy as np

In [ ]:
# db 연결
endpoint = 'dao.c51deksujiip.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

In [ ]:
# df_main 생성 : maindata
sql = "SELECT * FROM {}".format('MainData')
df_main=pd.read_sql(sql,db_connection)
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                497 non-null    int64  
 1   위도                497 non-null    float64
 2   경도                497 non-null    float64
 3   생태통로_유형           497 non-null    int64  
 4   차선_수              497 non-null    int64  
 5   등산객_이용빈도          497 non-null    int64  
 6   생태통로_효율성          497 non-null    int64  
 7   이용확인종_수           497 non-null    int64  
 8   폭(m)              270 non-null    float64
 9   연장(m)             271 non-null    float64
 10  높이(m)             21 non-null     float64
 11  개방도               22 non-null     float64
 12  경사도               497 non-null    object 
 13  주변 로드킬 빈도         497 non-null    int64  
 14  산책로까지의 최단 거리(km)  497 non-null    float64
 15  농가까지의 거리(km)      497 non-null    float64
 16  도로 최고제한속도(km/h)   497 non-null    int64  
 1

In [ ]:
# df_main을 유형에 따라 df_overpass(육교형)와 df_tunnel(터널형)로 분리
df_overpass=df_main[df_main['생태통로_유형']==0]
df_tunnel=df_main[(df_main['생태통로_유형']==1) | (df_main['생태통로_유형']==2)]

In [ ]:
# df_overpass에 불필요한 '높이(m)', '개방도' 열 제거
df_overpass.drop(columns=['높이(m)','개방도'],axis=1,inplace=True)

In [ ]:
df_overpass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 496
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                300 non-null    int64  
 1   위도                300 non-null    float64
 2   경도                300 non-null    float64
 3   생태통로_유형           300 non-null    int64  
 4   차선_수              300 non-null    int64  
 5   등산객_이용빈도          300 non-null    int64  
 6   생태통로_효율성          300 non-null    int64  
 7   이용확인종_수           300 non-null    int64  
 8   폭(m)              249 non-null    float64
 9   연장(m)             249 non-null    float64
 10  경사도               300 non-null    object 
 11  주변 로드킬 빈도         300 non-null    int64  
 12  산책로까지의 최단 거리(km)  300 non-null    float64
 13  농가까지의 거리(km)      300 non-null    float64
 14  도로 최고제한속도(km/h)   300 non-null    int64  
 15  주변 동물종 개수         300 non-null    int64  
 16  주변동물 출현빈도         300 non-null    int64  
 1

In [ ]:
df_tunnel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197 entries, 1 to 477
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                197 non-null    int64  
 1   위도                197 non-null    float64
 2   경도                197 non-null    float64
 3   생태통로_유형           197 non-null    int64  
 4   차선_수              197 non-null    int64  
 5   등산객_이용빈도          197 non-null    int64  
 6   생태통로_효율성          197 non-null    int64  
 7   이용확인종_수           197 non-null    int64  
 8   폭(m)              21 non-null     float64
 9   연장(m)             22 non-null     float64
 10  높이(m)             21 non-null     float64
 11  개방도               22 non-null     float64
 12  경사도               197 non-null    object 
 13  주변 로드킬 빈도         197 non-null    int64  
 14  산책로까지의 최단 거리(km)  197 non-null    float64
 15  농가까지의 거리(km)      197 non-null    float64
 16  도로 최고제한속도(km/h)   197 non-null    int64  
 1

In [ ]:
# df → db
table_name1 = 'MainData_overpass'
table_name2 = 'MainData_tunnel'
df_overpass.to_sql(name=table_name1, con=db_connection, if_exists='replace', index=False)
df_tunnel.to_sql(name=table_name2, con=db_connection, if_exists='replace', index=False)